<a href="https://colab.research.google.com/github/88FaHaD/ML/blob/main/mobilenetssdwithifrozengraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3075/3075), done.
remote: Total 4084 (delta 1189), reused 2900 (delta 949), pack-reused 0
Receiving objects: 100% (4084/4084), 44.61 MiB | 25.55 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.


In [3]:

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━

In [4]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2024-03-08 17:45:35.017274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 17:45:35.017326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 17:45:35.018923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 17:45:36.080414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2024-03-08 17:45:40.305656: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting beca

In [5]:
! pip install kaggle

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
! mkdir ~/.kaggle

In [8]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download gpiosenka/100-bird-species

100% 1.95G/1.96G [00:21<00:00, 154MB/s]
100% 1.96G/1.96G [00:21<00:00, 96.2MB/s]


In [11]:
import zipfile

zip_file_path = "/content/100-bird-species.zip"

extract_to_directory = "/content/bird_species_data"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (160, 160)
BATCH_SIZE = 32

train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir=   '/content/bird_species_data/test'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("validdation images:")
valid_data = train_datagen.flow_from_directory(valid_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

Training images:
Found 84635 images belonging to 525 classes.
validdation images:
Found 2625 images belonging to 525 classes.


In [13]:
!pip install tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.8 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844486 sha256=ed2d00459199ba8f1b9c0e58ef1453928d2ef18c7305f6db3cdf62b7464aa815
  Stored in directory: /root/.cache/pip/wheels/8a/55/68/c084bc2cd93c41fd8f7e2ef9e6bbcb2c35a3e4b49e42044d02
Successfully built tensorflow-object-detection-api


In [14]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (160, 160)
BATCH_SIZE = 32

train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir=   '/content/bird_species_data/test'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("validdation images:")
valid_data = train_datagen.flow_from_directory(valid_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

Training images:
Found 84635 images belonging to 525 classes.
validdation images:
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [15]:
train_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [16]:
from tensorflow.keras import layers, Model

In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(525, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 [==============================] - 0s 0us/step


In [18]:
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(train_data,
                    epochs=10,
                    validation_data=valid_data)

Epoch 1/10
2645/2645 [==============================] - 270s 89ms/step - loss: 2.3127 - accuracy: 0.4732 - val_loss: 9.5736 - val_accuracy: 0.1227
Epoch 2/10
2645/2645 [==============================] - 235s 89ms/step - loss: 0.9719 - accuracy: 0.7427 - val_loss: 2.2331 - val_accuracy: 0.5364
Epoch 3/10
2645/2645 [==============================] - 227s 86ms/step - loss: 0.7132 - accuracy: 0.8063 - val_loss: 1.9978 - val_accuracy: 0.6000
Epoch 4/10
2645/2645 [==============================] - 226s 86ms/step - loss: 0.5713 - accuracy: 0.8402 - val_loss: 1.2324 - val_accuracy: 0.7204
Epoch 5/10
2645/2645 [==============================] - 228s 86ms/step - loss: 0.4807 - accuracy: 0.8645 - val_loss: 1.3672 - val_accuracy: 0.7177
Epoch 6/10
2645/2645 [==============================] - 224s 85ms/step - loss: 0.4071 - accuracy: 0.8822 - val_loss: 0.7767 - val_accuracy: 0.7977
Epoch 7/10
2645/2645 [==============================] - 225s 85ms/step - loss: 0.3579 - accuracy: 0.8955 - val_loss: 0

In [20]:
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

Found 2625 images belonging to 525 classes.


In [ ]:
model.evaluate(test_data)

31/83 [==========>...................] - ETA: 2s - loss: 0.5912 - accuracy: 0.8387

#Creating the frozen graph